# 酒店预订数据分析
使用pyecharts库做可视化分析
## 加载库

In [50]:
import pandas as pd
import numpy as np
from pyecharts import options as opts
from pyecharts.charts import Bar
from pyecharts.globals import ThemeType
from pyecharts.charts import Map
from pyecharts.charts import Line
from pyecharts.charts import Pie
import calendar
import pycountry

## 数据加载及预览
### 列名解释
* **hotel** City Hotel-城市酒店 Resort Hotel-度假酒店
* **is_canceled** 预定是否取消 1-取消 0未取消
* **lead_time** 进入PMS（酒店物业管理系统）订房日期与到店日期间隔
* **arrival_date_year** 到店年份
* **arrival_date_month** 到店月份
* **arrival_date_week_number** 到店星期是一年中的第几个星期
* **arrival_date_day_of_month** 到店日期
* **stays_in_weekend_nights** 周末入住的夜数
* **stays_in_week_nights** 工作日入住的夜数
* **adults** 成年人数量
* **children** 孩子数量
* **babies** 婴儿数量
* **meal** 餐食规格 SC/undefine-未订餐 BB-早餐 HB-早餐加中餐或午餐 BB-全餐
* **country** 来自哪个国家
* **market_segment** 细分市场名称 TO-旅游批发商 TA-旅游零售商
* **distribution_channel** 预定渠道
* **is_repeated_guest** 预定名是否来自于重复的客人 1为是 0为否
* **previous_cancellations** 客户在当前预定前取消预订的次数
* **previous_bookings_not_canceled** 客户在当前预定前未取消预订的次数
* **reserved_room_type** 预定房型
* **assigned_room_type** 分配房型（预定房型已满或其他原因）
* **booking_changes** 从预定到入住订单的修改次数
* **deposit_type** 押金类型 No Deposit0没有押金 Non Refund-押金不退还 Refundable-押金可退还
* **agent** 旅行社ID
* **company** 预定公司的ID
* **days_in_waiting_list** 订单确认前在等待列表中的天数
* **customer_type** 顾客类型 Contract-合同 Group-团体 Transient-临时 Transient-party-与其他临时订单相关
* **adr** 平均每晚入住花费
* **required_car_parking_spaces** 客户要求的停车位数量
* **total_of_special_requests** 特殊要求数量
* **reservation_status** 订单状态 Canceled-取消 Check-Out-已退房完成订单 No-Show-未入住
* **reservation_status_date** 设置订单最后状态的日期

In [51]:
df = pd.read_csv('hotel_bookings.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119390 entries, 0 to 119389
Data columns (total 32 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   hotel                           119390 non-null  object 
 1   is_canceled                     119390 non-null  int64  
 2   lead_time                       119390 non-null  int64  
 3   arrival_date_year               119390 non-null  int64  
 4   arrival_date_month              119390 non-null  object 
 5   arrival_date_week_number        119390 non-null  int64  
 6   arrival_date_day_of_month       119390 non-null  int64  
 7   stays_in_weekend_nights         119390 non-null  int64  
 8   stays_in_week_nights            119390 non-null  int64  
 9   adults                          119390 non-null  int64  
 10  children                        119386 non-null  float64
 11  babies                          119390 non-null  int64  
 12  meal            

## 分析问题
* 旅客来自哪里
* 酒店平均房费
* 酒店入住高峰期
* 旅客平均入住多久
* 预定取消率及相关性因素

## 数据预处理
children,country,agent,company列含有空值，用0替代

In [52]:
df = df.fillna(0)
#预定餐食Undefined的数据转为SC
df = df.replace('Undefined','SC')
#删除部分订单成人孩子婴儿都是0的数据。
df = df.drop(df[df['children']+df['adults']+df['babies'] ==0].index)

In [53]:
#部分订单成人孩子婴儿都是0的删除数据。
df.drop(df[df['children']+df['adults']+df['babies'] ==0].index)

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,...,No Deposit,0.0,0.0,0,Transient,0.00,0,0,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,...,No Deposit,0.0,0.0,0,Transient,0.00,0,0,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,...,No Deposit,0.0,0.0,0,Transient,75.00,0,0,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,...,No Deposit,304.0,0.0,0,Transient,75.00,0,0,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,...,No Deposit,240.0,0.0,0,Transient,98.00,0,1,Check-Out,2015-07-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119385,City Hotel,0,23,2017,August,35,30,2,5,2,...,No Deposit,394.0,0.0,0,Transient,96.14,0,0,Check-Out,2017-09-06
119386,City Hotel,0,102,2017,August,35,31,2,5,3,...,No Deposit,9.0,0.0,0,Transient,225.43,0,2,Check-Out,2017-09-07
119387,City Hotel,0,34,2017,August,35,31,2,5,2,...,No Deposit,9.0,0.0,0,Transient,157.71,0,4,Check-Out,2017-09-07
119388,City Hotel,0,109,2017,August,35,31,2,5,2,...,No Deposit,89.0,0.0,0,Transient,104.40,0,0,Check-Out,2017-09-07


### 住客都来自哪里？

In [54]:
#已经入住并退房的住客
no_cancel_df = df.loc[df['reservation_status']=='Check-Out']
no_cancel_df.country.value_counts()
#df转换成列表
country_list_for_pyecharts = no_cancel_df.country.value_counts().index.tolist()
country_guests_for_pyecharts = no_cancel_df.country.value_counts().tolist()

In [55]:
#pyecharts饼图
guests_from_pie = (
    Pie(init_opts=opts.InitOpts(theme=ThemeType.ROMA))
.add('',[list(z) for z in zip(country_list_for_pyecharts, country_guests_for_pyecharts)],label_opts=opts.LabelOpts(position='inside',formatter="{b}\n{c}\n{d}%"))
.set_global_opts(
        legend_opts=opts.LegendOpts(type_="scroll", pos_left="80%", orient="vertical"),
        title_opts=opts.TitleOpts(title="住客国籍"),
        tooltip_opts=opts.TooltipOpts(formatter="{b}:{c}\n{d}%")
    )
)
guests_from_pie.render_notebook()

In [56]:
#国家iso代码转换为国家全名
def tran_country_code(x):
    try:
        return pycountry.countries.get(alpha_3=x).name
    except:
        return 'other'
c = (
    Map(init_opts=opts.InitOpts(theme=ThemeType.ROMA))
    .add("", [list(z) for z in zip(list(map(tran_country_code,country_list_for_pyecharts)), country_guests_for_pyecharts)], "world")
    .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
    .set_global_opts(
        title_opts=opts.TitleOpts(title="住客国籍"),
        visualmap_opts=opts.VisualMapOpts(max_=21000),
    )
)
c.render_notebook()

住客主要来自于葡萄牙及周边欧洲国家
## 酒店平均房费

In [57]:
rh = no_cancel_df.loc[no_cancel_df['hotel'] == 'Resort Hotel']
ch = no_cancel_df.loc[no_cancel_df['hotel'] == 'City Hotel']
print('度假酒店平均房价：',rh['adr'].sum()/len(rh))
print('城市酒店平均房价：',ch['adr'].sum()/len(ch))

度假酒店平均房价： 90.82252670515436
城市酒店平均房价： 106.03614117698118


房价趋势

In [58]:
# 月份英文改为数字
rh['arrival_date_month'] = rh['arrival_date_month'].apply(lambda x:list(calendar.month_name).index(x))
ch['arrival_date_month'] = ch['arrival_date_month'].apply(lambda x:list(calendar.month_name).index(x))

D:\Users\scofield\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
D:\Users\scofield\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [59]:
adr_month = (
    Line()
    .add_xaxis(list(map(str,list(range(1,13)))))
    .add_yaxis("度假酒店", rh.sort_values('arrival_date_month').groupby('arrival_date_month').adr.mean().tolist(),is_smooth=True,label_opts=opts.LabelOpts(is_show=False))
    .add_yaxis("城市酒店", ch.sort_values('arrival_date_month').groupby('arrival_date_month').adr.mean().tolist(),is_smooth=True,label_opts=opts.LabelOpts(is_show=False))
    .set_global_opts(
        title_opts=opts.TitleOpts(title="房价趋势图"),
        xaxis_opts=opts.AxisOpts(type_="category",name='月份', boundary_gap=False),
        yaxis_opts=opts.AxisOpts(
            type_="value",
            name="房价（欧元）",
            splitline_opts=opts.SplitLineOpts(is_show=True),
        ),
        tooltip_opts=opts.TooltipOpts(trigger="axis"),
        toolbox_opts=opts.ToolboxOpts(is_show=True),
    )
)
adr_month.render_notebook()

In [60]:
check_in_counts_month = (
    Line()
    .add_xaxis(list(map(str,list(range(1,13)))))
    .add_yaxis("度假酒店成人", rh[['arrival_date_day_of_month','adults']].groupby('arrival_date_day_of_month').sum().sum(axis=1).tolist(),is_smooth=True,label_opts=opts.LabelOpts(is_show=False))
    .add_yaxis("度假酒店未成年", rh[['arrival_date_day_of_month','children','babies']].groupby('arrival_date_day_of_month').sum().sum(axis=1).tolist(),is_smooth=True,label_opts=opts.LabelOpts(is_show=False))
    .add_yaxis("城市酒店成人", ch[['arrival_date_day_of_month','adults']].groupby('arrival_date_day_of_month').sum().sum(axis=1).tolist(),is_smooth=True,label_opts=opts.LabelOpts(is_show=False))
    .add_yaxis("城市酒店未成年", ch[['arrival_date_day_of_month','children','babies']].groupby('arrival_date_day_of_month').sum().sum(axis=1).tolist(),is_smooth=True,label_opts=opts.LabelOpts(is_show=False))
    .set_global_opts(
        title_opts=opts.TitleOpts(title="每月平均入住人数"),
        xaxis_opts=opts.AxisOpts(type_="category",name='月份', boundary_gap=False),
        yaxis_opts=opts.AxisOpts(
            type_="value",
            name="人数",
            splitline_opts=opts.SplitLineOpts(is_show=True),
        ),
        tooltip_opts=opts.TooltipOpts(trigger="axis"),
        toolbox_opts=opts.ToolboxOpts(is_show=True),
    )
)
check_in_counts_month.render_notebook()

## 入住率比较

In [61]:
df
check_in_rate = df

In [62]:
full_rh = df.loc[df['hotel'] == 'Resort Hotel']
full_ch = df.loc[df['hotel'] == 'City Hotel']
rh_check_in_rate =(full_rh['reservation_status'] == 'Check-Out').sum()/(full_rh['reservation_status'] == 'Check-Out').count()
ch_check_in_rate = (full_ch['reservation_status'] == 'Check-Out').sum()/(full_ch['reservation_status'] == 'Check-Out').count()

## 预测取消

In [67]:
cancel_corr = df.corr()["is_canceled"]
cancel_corr.abs().sort_values(ascending=False)[1:]

lead_time                         0.292876
total_of_special_requests         0.234877
required_car_parking_spaces       0.195701
booking_changes                   0.144832
previous_cancellations            0.110139
is_repeated_guest                 0.083745
company                           0.083594
adults                            0.058182
previous_bookings_not_canceled    0.057365
days_in_waiting_list              0.054301
agent                             0.046770
adr                               0.046492
babies                            0.032569
stays_in_week_nights              0.025542
arrival_date_year                 0.016622
arrival_date_week_number          0.008315
arrival_date_day_of_month         0.005948
children                          0.004851
stays_in_weekend_nights           0.001323
Name: is_canceled, dtype: float64

In [78]:
from sklearn.model_selection import train_test_split, KFold, cross_validate, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import eli5 # Feature importance evaluation

D:\Users\scofield\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
D:\Users\scofield\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)


In [80]:
#数值特征
num_features = ["lead_time","arrival_date_week_number","arrival_date_day_of_month",
                "stays_in_weekend_nights","stays_in_week_nights","adults","children",
                "babies","is_repeated_guest", "previous_cancellations",
                "previous_bookings_not_canceled","agent","company",
                "required_car_parking_spaces", "total_of_special_requests", "adr"]
#非数值特征
cat_features = ["hotel","arrival_date_month","meal","market_segment",
                "distribution_channel","reserved_room_type","deposit_type","customer_type"]
features = num_features + cat_features
X = df.drop(["is_canceled"], axis=1)[features]
y = df["is_canceled"]

# preprocess numerical feats:
# for most num cols, except the dates, 0 is the most logical choice as fill value
# and here no dates are missing.
num_transformer = SimpleImputer(strategy="constant")

# Preprocessing for categorical features:
cat_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="constant", fill_value="Unknown")),
    ("onehot", OneHotEncoder(handle_unknown='ignore'))])

# Bundle preprocessing for numerical and categorical features:
preprocessor = ColumnTransformer(transformers=[("num", num_transformer, num_features),
                                               ("cat", cat_transformer, cat_features)])

In [96]:
# define models to test:
base_models = [("DT_model", DecisionTreeClassifier(random_state=42)),
               ("RF_model", RandomForestClassifier(random_state=42,n_jobs=-1)),
               ("LR_model", LogisticRegression(random_state=42,n_jobs=-1)),
               ("XGB_model", XGBClassifier(random_state=42, n_jobs=-1))]

# split data into 'kfolds' parts for cross validation,
# use shuffle to ensure random distribution of data:
kfolds = 4 # 4 = 75% train, 25% validation
split = KFold(n_splits=kfolds, shuffle=True, random_state=42)

# Preprocessing, fitting, making predictions and scoring for every model:
for name, model in base_models:
    # pack preprocessing of data and the model in a pipeline:
    model_steps = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)])
    
    # get cross validation score for each model:
    cv_results = cross_val_score(model_steps, 
                                 X, y, 
                                 cv=split,
                                 scoring="accuracy",
                                 n_jobs=-1)
    # output:
    min_score = round(min(cv_results), 4)
    max_score = round(max(cv_results), 4)
    mean_score = round(np.mean(cv_results), 4)
    std_dev = round(np.std(cv_results), 4)
    print(f"{name} cross validation accuarcy score: {mean_score} +/- {std_dev} (std) min: {min_score}, max: {max_score}")
    print(cv_results)
    #pipeline.fit(X_train, y_train)
    #preds = pipeline.predict(X_valid)
    #score = accuracy_score(y_valid, preds)
    #print(f"{name} accuracy_score: {round(score, 4)}")
    

DT_model cross validation accuarcy score: 0.8256 +/- 0.0013 (std) min: 0.8243, max: 0.8276
[0.82431299 0.82468208 0.82756191 0.82595128]
RF_model cross validation accuarcy score: 0.8667 +/- 0.0005 (std) min: 0.8658, max: 0.867
[0.86689259 0.86692615 0.86581437 0.86702235]
LR_model cross validation accuarcy score: 0.7946 +/- 0.0027 (std) min: 0.7912, max: 0.798
[0.7928732  0.79639634 0.79796658 0.79122207]
XGB_model cross validation accuarcy score: 0.8468 +/- 0.0011 (std) min: 0.8455, max: 0.8483
[0.84612287 0.84733081 0.84833233 0.84548017]


In [98]:
split

KFold(n_splits=4, random_state=42, shuffle=True)